# Using Streamlit and Solara inside Notebooks Hub!

**Objective:** This workshop is to teach users to create an example Streamlit and Solara application using Notebooks Hub.

**Task:** Users will create an application generating a scatter plot of a sample dataset (`iris`) or by uploading their own file.


## Setup

### Launch VS Code Instance using Notebooks Hub

For documentation on Notebooks Hub, visit [here](https://polusai.github.io/notebooks-hub/index.html).

- Open Notebooks Hub: [login here](https://notebooks.scb-ncats.io/home)
- Launch VS Code
    - Method 1: Quick launch
    - Method 2: Create new instance
        - Go to **Servers** tab
        - Select **Create New Server** to load wizard
        - Select **Select Dashboard** tab to pick appropriate type
        - Select **Select File/Folder**
        - Navigate to `.py` file
        - Select appropriate hardware (**Shared Medium** is generally sufficient)
        - Select appropriate dependencies module (load none, a custom environment, or `python-data-science-0.1.8`) 
        - Add server metadata details
        - Check **Launch** box
        - Click **Create Server** button

### Create Virtual Environment

- Open/add `work` folder to workspace
- Create virtual environment using terminal using the following steps

1. Create virtual environment `test-env`

In [ ]:
python -m venv test-env

2. Activate virtual environment `test-env`

In [ ]:
source ./test-env/bin/activate

3. Install necessary packages inside `test-env`

In [ ]:
pip install solara plotly pandas        # for solara
pip install streamlit                   # for streamlit, also install plotly and pandas if not installing solara

### Create Help File For Prefix Extraction (Solara Only)

- Create new file named `prefix.py`
- Add the following code:

In [ ]:
# simple automated script for proper port forwarding of Solara app when launched inside Notebooks Hub VSCode instance
import os

# extract jupyterhub service prefix and modify for solara service prefix
jhsp = os.environ['JUPYTERHUB_SERVICE_PREFIX']
ssp = jhsp + 'proxy/8765'

# output the desired path to use
print(ssp)

## Solara

For documentation on Solara, visit the [official site](https://solara.dev/api) or the [Notebooks Hub documentation](https://polusai.github.io/notebooks-hub/user/solara-intro.html#).

### Create App

- Create new file named `sol.py` (rename as desired)
- Add code to `sol.py`

The following code was used in the workshop:

In [ ]:
# import dependencies
import solara
import plotly.express as px
import pandas as pd
from typing import Optional, cast
from solara.components.file_drop import FileDrop

# load sample dataset
df_sample = px.data.iris()

# state management
class State:
    # set reactive variables to use with widgets
    color = solara.reactive(cast(Optional[str], None))
    x = solara.reactive(cast(Optional[str], None))
    y = solara.reactive(cast(Optional[str], None))
    df = solara.reactive(cast(Optional[pd.DataFrame], None))

    # assign initial values when loading sample dataset
    @staticmethod
    def load_sample():
        State.x.value = str("sepal_length")
        State.y.value = str("sepal_width")
        State.color.value = str("species")
        State.df.value = df_sample
    
    # assign initial values when uploading custom dataset
    @staticmethod
    def load_from_file(file):
        df = pd.read_csv(file["file_obj"])
        State.x.value = str(df.column[0])
        State.y.value = str(df.column[1])
        State.color.value = str(df.column[2])
        State.df.value = df
    
    # clear stored dataframe upon reset
    @staticmethod
    def reset():
        State.df.value = None

# define app's main component
@solara.component
def Page():
    # initialize variable from state
    df = State.df.value

    # name app
    with solara.AppBarTitle():
        solara.Text("Solara Demo")
    
    # create sidebar panel
    with solara.Sidebar():
        with solara.Column():
            # put buttons in a row
            with solara.Row():
                solara.Button("Sample dataset", color="primary", text=True, outlined=True, on_click=State.load_sample, disabled=df is not None)
                solara.Button("Clear dataset", color="primary", text=True, outlined=True, on_click=State.reset)
            FileDrop(on_file=State.load_from_file, on_total_progress=lambda *args: None, label="Drag file here")
        # put selection widgets in a column inside sidebar panel
        if df is not None:
            columns = list(map(str, df.columns))
            solara.Select("X-axis", values=columns, value=State.x)
            solara.Select("Y-axis", values=columns, value=State.y)
            solara.Select("Color", values=columns, value=State.color)
    
    # create scatter plot
    if df is not None:
        if State.x.value and State.y.value:
            fig = px.scatter(
                df,
                State.x.value,
                State.y.value,
                color = State.color.value
            )
            solara.FigurePlotly(fig)
            solara.DataFrame(df, items_per_page=10)
        else:
            solara.Warning("Select x and y axes")
    else:
        solara.Info("No data loaded")

# app layout
@solara.component
def Layout(children):
    route, routes = solara.use_route()
    return solara.AppLayout(children=children)

### Launch App using Terminal

#### Extract prefix from help file created during setup

- Inside terminal, run the following line to output identifier (similar to `/user/<email-id>/<server-ID>/proxy/8765`)

In [ ]:
python prefix.py

#### Assign Solara App Variables

- Inside terminal, assign Solara variables (when launch inside VS Code only, this is baked into launching Solara as dashboard app)

In [ ]:
export SOLARA_SERVICE_PREFIX=<ssp>       # replace `<ssp>` with the output from `python prefix.py`

In [ ]:
export SOLARA_APP=sol.py                # replace `sol.py` with desired app filename if renamed

#### Launch Solara App

- Inside terminal, run the following line to launch Solara app with starlette using ASGI server uvicorn

In [ ]:
SOLARA_APP=$SOLARA_APP uvicorn --workers 1 --root-path $SOLARA_SERVICE_PREFIX --host 0.0.0.0 --port 8765 solara.server.starlette:app

#### Exit App

- To exit, type `Ctrl-C` inside the terminal

### Import App Components Into Jupyter Notebook

- Create new Jupyter notebook named `sol_nb.ipynb` (rename as desired)
- Select `test-env` kernel
- Create code cell and import desired Solara component from `sol.py`

In [ ]:
from sol import Page        # do not add .py extension here

- display imported component using `display()`

In [ ]:
display(Page())

## Streamlit

For documentation on Streamlit, visit the [official site](https://docs.streamlit.io/library/api-reference) or the [Notebooks Hub documentation](https://polusai.github.io/notebooks-hub/user/streamlit_1-intro.html).

- Create new file named `app.py` (rename as desired)
- Add code to `app.py`

The following code was used in the workshop:

In [ ]:
# import dependencies
import streamlit as st
import pandas as pd
import plotly.express as px

# set up tabs
tab1, tab2 = st.tabs(["Sample: Iris", "Upload Data"])

# tab for sample dataset
with tab1:
    # load data
    iris = px.data.iris()

    # editable dataframe
    with st.expander("See & Edit Data"):
        edited_df = st.data_editor(
            iris, # source dataframe
            num_rows="dynamic", # enable addition and removal of rows
            hide_index=True
        )

    # add widgets for x-axis, y-axis, and color selection
    x_ax = st.selectbox(
        "Select X-axis",
        iris.columns,
        index=0
    )

    y_ax = st.selectbox(
        "Select Y-axis",
        iris.columns,
        index=1
    )

    color = st.selectbox(
        "Select Color",
        iris.columns,
        index=4
    )

    # plotly figure
    fig = px.scatter(
            edited_df,
            x = x_ax,
            y = y_ax,
            color = color)
    st.plotly_chart(fig, use_container_width=True)

# tab for uploaded file
with tab2:
    # upload data
    uploaded_file = st.file_uploader("Choose a file")
    if uploaded_file is not None:
        data = pd.read_csv(uploaded_file)

        # editable dataframe
        with st.expander("See & Edit Data"):
            edited_data = st.data_editor(
                data, # source dataframe
                num_rows="dynamic", # enable addition and removal of rows
                hide_index=True
            )

        # add widgets for x- axis, y-axis, and color selection
        x_ax2 = st.selectbox(
            "Select X-axis",
            edited_data.columns,
            index=0
        )

        y_ax2 = st.selectbox(
            "Select Y-axis",
            edited_data.columns,
            index=1
        )

        color2 = st.selectbox(
            "Select Color",
            edited_data.columns,
            index=None
        )

        # plotly figure
        fig2 = px.scatter(
                edited_data,
                x = x_ax2,
                y = y_ax2,
                color = color2)
        st.plotly_chart(fig2, use_container_width=True)


### Launch App using Terminal

- Ensure proper working directory in terminal (use `cd` to change directory if necessary)
- Run the following code to launch Streamlit app

In [ ]:
streamlit run app.py            # rename `app.py` with desired filename if renamed

### Exit App

- To exit, type `Ctrl-C` inside the terminal

## Launch as Dashboard App inside Notebooks Hub

For documentation on Notebooks Hub, visit [here](https://polusai.github.io/notebooks-hub/index.html).

- Open Notebooks Hub: [login here](https://notebooks.scb-ncats.io/home)
- Go to **Servers** tab
- Select **Create New Server** to load wizard
- Select **Select Dashboard** tab to pick appropriate type
- Select **Select File/Folder**
- Navigate to `.py` file
- Select appropriate hardware (**Shared Medium** is generally sufficient)
- Select `python-data-science-0.1.8` module
- Add server metadata details
- Check **Launch** box
- Click **Create Server** button